In [33]:
import pandas as pd
import numpy as np

df = pd.read_csv('../../Data/data_after_predicting_anaemia_values.csv',sep=',')


In [34]:
df['Allogeneic Blood Transfusion'].describe()

count     26860
unique        2
top          NO
freq      23581
Name: Allogeneic Blood Transfusion, dtype: object

In [35]:
df[df['Allogeneic Blood Transfusion'] == 'YES'].shape

(3279, 20)

In [36]:
df_allo = df[df['Allogeneic Blood Transfusion'] == 'YES']

In [37]:
del df_allo['Unnamed: 0']
del df_allo['Masked FIN']
del df_allo['Sequence No.']
del df_allo['Duration of Surgery (hh:mm).1']
del df_allo['SN - BM - PRBC Ordered']
del df_allo['EBL']
del df_allo['ResultAfterSurgery']

del df_allo['SN - BM - Red Blood Cells']
del df_allo['SN - BM - Cryoprecipitate']
del df_allo['SN - BM - Platelets']

In [38]:
y = df_allo['SN - BM - Fresh Frozen Plasma']
del  df_allo['SN - BM - Fresh Frozen Plasma']
X = df_allo

In [39]:
y.describe()

count    3279.000000
mean        2.642432
std         5.126772
min         0.000000
25%         0.000000
50%         0.000000
75%         3.000200
max        55.004500
Name: SN - BM - Fresh Frozen Plasma, dtype: float64

In [40]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [17]:
fig = plt.figure()
ax = fig.add_subplot(111)
sns.distplot(y)
plt.savefig('../../Visualizations/FFP/histogram_ffp.png')

In [18]:
y[y == 0.0].shape

(1658,)

In [41]:
corr = df[df['Allogeneic Blood Transfusion'] == 'YES'].corr(method='pearson')
corr_with_target = corr.ix[-1][:-1]
corr_with_target

Unnamed: 0                         0.051539
age                                0.045584
Sequence No.                      -0.022239
SN - BM - Pre-Op INR              -0.113581
SN - BM - Pre-Op Platelet Count    0.088900
SN - BM - PRBC Ordered            -0.075778
SN - BM - Red Blood Cells          0.015197
SN - BM - Fresh Frozen Plasma     -0.151741
SN - BM - Platelets               -0.125514
SN - BM - Cryoprecipitate         -0.115198
ResultsBeforeSurgery               0.235837
Name: ResultAfterSurgery, dtype: float64

In [42]:
X = pd.get_dummies(X,drop_first=True)

In [43]:
X.shape, y.shape

((3279, 474), (3279,))

In [44]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.10,random_state=42)

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor()
param_grid = {'n_estimators': [100,150,200]}

In [46]:
grid = GridSearchCV(reg,param_grid,verbose=10)
grid.fit(X_train,y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] n_estimators=100 ................................................
[CV] ................. n_estimators=100, score=0.393565, total=   5.2s
[CV] n_estimators=100 ................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.3s remaining:    0.0s


[CV] ................. n_estimators=100, score=0.333552, total=   4.7s
[CV] n_estimators=100 ................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   10.1s remaining:    0.0s


[CV] ................. n_estimators=100, score=0.414296, total=   4.7s
[CV] n_estimators=150 ................................................


...

[CV] ................. n_estimators=200, score=0.323207, total=   9.5s
[CV] n_estimators=200 ................................................


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   57.5s remaining:    0.0s


[CV] ................. n_estimators=200, score=0.419650, total=   9.5s


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.1min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 150, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [49]:
y_pred = grid.best_estimator_.predict(X_test)

In [50]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_pred)

17.74440722330079

In [52]:
df.head()

,Unnamed: 0,Masked FIN,age,Sequence No.,SURG_PROCEDURE,Duration of Surgery (hh:mm).1,SURGICAL_SPECIALTY,Surgeon Hash Name,PATIENT_TYPE,SN - BM - Pre-Op INR,SN - BM - Pre-Op Platelet Count,SN - BM - PRBC Ordered,Allogeneic Blood Transfusion,SN - BM - Red Blood Cells,SN - BM - Fresh Frozen Plasma,SN - BM - Platelets,SN - BM - Cryoprecipitate,ResultsBeforeSurgery,ResultAfterSurgery,EBL
0,0,A10454,36,11978,ACHILLES TENDON REPAIR,1:04,ORTHOPAEDIC SURGERY,DR. 281,DAY SURGERY,0.000,0.00,0.0,NO,0.0,0.0,0.0,0.0,15.6063,14.166127,0
1,1,A492,48,184,ANAL CONDYLOMA EXCISION,0:23,COLORECTAL SURGERY,DR. 236,OBSERVATION,0.000,0.00,0.0,NO,0.0,0.0,0.0,0.0,14.5018,13.498745,0
2,2,A24199,25,28931,ANAL FISTULOTOMY WITH SETON,0:20,COLORECTAL SURGERY,DR. 25,DAY SURGERY,0.904,288.09,0.0,NO,0.0,0.0,0.0,0.0,17.0057,15.418479,0
3,3,A13935,29,16255,ANAL FISTULOTOMY WITH SETON,0:19,COLORECTAL SURGERY,DR. 236,DAY SURGERY,0.000,0.00,0.0,NO,0.0,0.0,0.0,0.0,13.3047,12.610660,0
4,4,A7660,56,8755,ANAL FISTULOTOMY WITH SETON,1:54,COLORECTAL SURGERY,DR. 25,INPATIENT,0.000,0.00,0.0,NO,0.0,0.0,0.0,0.0,11.6062,10.206200,0


In [65]:
################# Approach: 2 with lesser number of columns #############
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=20, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [66]:
X_pca = pca.fit_transform(X)

In [67]:
X_train,X_test,y_train,y_test = train_test_split(X_pca,y,test_size=0.10,random_state=42)

In [68]:
reg = RandomForestRegressor()
param_grid = {'n_estimators':[100,150,200]}
grid = GridSearchCV(reg,param_grid,verbose=10)
grid.fit(X_train,y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] n_estimators=100 ................................................
[CV] ................. n_estimators=100, score=0.322860, total=   3.0s
[CV] n_estimators=100 ................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.1s remaining:    0.0s


[CV] ................. n_estimators=100, score=0.367294, total=   2.9s
[CV] n_estimators=100 ................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.1s remaining:    0.0s


[CV] ................. n_estimators=100, score=0.413257, total=   2.8s
[CV] n_estimators=150 ................................................


...

[CV] ................. n_estimators=200, score=0.358088, total=   5.8s
[CV] n_estimators=200 ................................................


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   34.3s remaining:    0.0s


[CV] ................. n_estimators=200, score=0.413706, total=   5.4s


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   39.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   39.9s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 150, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [72]:
df_allo['SURGICAL_SPECIALTY'].describe()

count                       3279
unique                        21
top       CARDIOTHORACIC SURGERY
freq                        1049
Name: SURGICAL_SPECIALTY, dtype: object